In [2]:
#!pip install pandasql 
from pandasql import sqldf 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [12]:
game_data = pd.read_excel(r'C:\Users\vasaa\data2\Data Challenge Master File.xlsx', sheet_name='Single Game Sales and Scans')
UCP_Scans = pd.read_excel(r'C:\Users\vasaa\data2\Data Challenge Master File.xlsx', sheet_name='UCP Scans')
UCP_Reservations = pd.read_excel(r'C:\Users\vasaa\data2\Data Challenge Master File.xlsx', sheet_name='UCP Reservations')
UCP_Member_Report = pd.read_excel(r'C:\Users\vasaa\data2\Data Challenge Master File.xlsx', sheet_name='UCP Member Report')


In [ ]:
Data_Dictionary = pd.read_excel('/content/Data Challenge Master File.xlsx', sheet_name='Data Dictionary')
team = pd.read_excel('/content/Data Challenge Master File.xlsx', sheet_name='Team Data')


In [5]:
pysqldf = lambda q: sqldf(q, globals())

# Do evening games (6:00pm or later) on average, have a higher scanned attendance than afternoon games (Before 6:00pm)?

In [1118]:
query1="""WITH game_attendance AS (
  SELECT 
    event_datetime,
    event_name,
    CASE 
      WHEN strftime('%H', event_datetime) < '18' THEN 'Afternoon' 
      ELSE 'Evening' 
    END as game_time,
    COUNT(scan_datetime) AS total_attendance
  FROM game_data
  GROUP BY event_datetime, event_name, game_time
)

SELECT 
  game_time,
  ROUND(AVG(total_attendance)) AS avg_attendance
FROM game_attendance
GROUP BY game_time;
"""

In [1119]:
pysqldf(query1)

,game_time,avg_attendance
0,Afternoon,1435.0
1,Evening,2132.0


## Also, provide average attendance per game by month

In [1148]:
query2="""SELECT 
  month,
  ROUND(AVG(total_attendance)) AS avg_attendance_per_game
FROM (
  SELECT 
    strftime('%m', event_datetime) AS month,
    event_name,
    COUNT(scan_datetime) AS total_attendance
  FROM 
    game_data
  GROUP BY 
    month, event_name
) subquery
GROUP BY 
  month;
"""

In [1149]:
pysqldf(query2)

,month,avg_attendance_per_game
0,01,2082.0
1,02,2682.0
2,03,2181.0
3,04,4363.0
4,10,1429.0
5,11,1632.0
6,12,1459.0


# Count the number of UCP members with reservations >=5 and scans >=3. 

In [728]:
query3="""SELECT COUNT(*) FROM(SELECT *
FROM UCP_Reservations AS r
JOIN UCP_Scans AS s ON r.EVENT_DATE_LOCAL = s.event_datetime AND r.SECTION = s.section_name AND r.ROW = s.row_name AND r.SEAT = s.seat_number
JOIN UCP_Member_Report AS m ON r.TENANT_MEMBER_NUMBER = m.TENANT_MEMBER_NUMBER
WHERE r.TENANT_MEMBER_NUMBER != '1000-000-0000'
GROUP BY r.TENANT_MEMBER_NUMBER
HAVING COUNT(DISTINCT r.TICKET_ID) >= 5 AND COUNT(DISTINCT s.scan_datetime) >= 3)
"""

In [729]:
pysqldf(query3)

,COUNT(*)
0,330


# Of these, how many UCP members attended every game they reserved?

In [730]:
query4='''
WITH Reservations AS (
    SELECT TENANT_MEMBER_NUMBER, COUNT(TICKET_ID) AS reservation_count
    FROM UCP_Reservations
    WHERE TENANT_MEMBER_NUMBER != '1000-000-0000'
    GROUP BY TENANT_MEMBER_NUMBER
),
Scans AS (
    SELECT UCP_Reservations.TENANT_MEMBER_NUMBER, COUNT(UCP_Scans.scan_datetime) AS scan_count
    FROM UCP_Reservations
    JOIN UCP_Scans ON UCP_Reservations.EVENT_DATE_LOCAL = UCP_Scans.event_datetime
        AND UCP_Reservations.SECTION = UCP_Scans.section_name
        AND UCP_Reservations.ROW = UCP_Scans.row_name
        AND UCP_Reservations.SEAT = UCP_Scans.seat_number
    WHERE UCP_Reservations.TENANT_MEMBER_NUMBER != '1000-000-0000'
    GROUP BY UCP_Reservations.TENANT_MEMBER_NUMBER
),
MemberData AS (
    SELECT Reservations.TENANT_MEMBER_NUMBER, reservation_count, scan_count
    FROM Reservations
    JOIN Scans ON Reservations.TENANT_MEMBER_NUMBER = Scans.TENANT_MEMBER_NUMBER
    WHERE reservation_count >= 5 AND scan_count >= 3
)

SELECT COUNT(*) AS total_members
FROM MemberData
WHERE reservation_count = scan_count;
'''

In [731]:
pysqldf(query4)

,total_members
0,26


# Find the average cost of a single game ticket for each game in the dataset provided.

In [732]:
query5 = '''
    SELECT event_name, AVG(price) AS avg_price
    FROM game_data
    GROUP BY event_name
'''

In [733]:
pysqldf(query5)

,event_name,avg_price
0,LA Clippers 2022-23 Game 1 - Phoenix 10/23/22,31.966790
1,LA Clippers 2022-23 Game 10 - Utah 11/21/22,19.911941
2,LA Clippers 2022-23 Game 11 - Denver 11/25/22,31.697708
3,LA Clippers 2022-23 Game 12 - Indiana 11/27/22,19.672161
4,LA Clippers 2022-23 Game 13 - Sacramento 12/3/22,23.143322
5,LA Clippers 2022-23 Game 14 - Boston 12/12/22,40.391410
6,LA Clippers 2022-23 Game 15 - Minnesota 12/14/22,17.140794
7,LA Clippers 2022-23 Game 16 - Phoenix 12/15/22,21.590062
8,LA Clippers 2022-23 Game 17 - Washington 12/17/22,21.790625
9,LA Clippers 2022-23 Game 18 - Charlotte 12/21/22,25.266808


# Then compare the total value if all UCP eligible games were purchased at their average single game price, against the UCP package price. Assume the UCP package price is $350. 

In [ ]:
# Excluding the 6 games which are not available in the UCP 

In [1146]:
query6 = '''SELECT ROUND(SUM(avg_price) - (78.283906 + 27.499471 + 83.415429 + 46.151941 + 84.752982 + 159.001586),0) as total_value
FROM(
    SELECT event_name, AVG(price) AS avg_price
    FROM game_data
    GROUP BY event_name) AS avg_CTE
'''

In [1147]:
pysqldf(query6)

,total_value
0,861.0
